In [155]:
import matplotlib.pyplot as plt
import pandas as pd

BASE_MUERTES_COVID = '../data/external/muertes-covid-por-pais.csv'
BASE_POBLACION = '../data/external/poblacion-por-pais.csv'

## Carregando as bases

In [156]:
# Carregando base 'muertes-covid-por-pais.csv'
df_muertes_covid = pd.read_csv(
    BASE_MUERTES_COVID,
    sep=',',
    header=0,
    names=['pais', 'fecha_inicio', 'fecha_fin', 'dias', 'semana', 'total_muertes_reportadas', 'total_muertes_reportadas_por_covid'],
    parse_dates=['fecha_inicio', 'fecha_fin']
    )
print(f"Dimensões da df_muertes_covid: {df_muertes_covid.shape}")

# Carregando base 'poblacion-por-pais.csv'
df_poblacion = pd.read_csv(
    BASE_POBLACION,
    sep=',',
    header=0,
    names=['pais', 'poblacion']
    )
print(f"Dimensões da df_poblacion: {df_poblacion.shape}")

Dimensões da df_muertes_covid: (1332, 7)
Dimensões da df_poblacion: (5, 2)


In [157]:
df_muertes_covid.head()

,pais,fecha_inicio,fecha_fin,dias,semana,total_muertes_reportadas,total_muertes_reportadas_por_covid
0,Mexico,2014-12-29,2015-01-04,7,1,15101,0
1,Mexico,2015-01-05,2015-01-11,7,2,15449,0
2,Mexico,2015-01-12,2015-01-18,7,3,14820,0
3,Mexico,2015-01-19,2015-01-25,7,4,14172,0
4,Mexico,2015-01-26,2015-02-01,7,5,13628,0


In [158]:
df_poblacion.head()

,pais,poblacion
0,Mexico,130262220
1,Peru,33359415
2,Chile,19212362
3,Brazil,213993441
4,Colombia,51265841


## Avaliando as bases

Devido à quantidade de registros, analisaremos somente a tabela 'df_muertes_covid'.

### Verificando nulos e valores incoerentes

In [159]:
df_muertes_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   pais                                1332 non-null   object        
 1   fecha_inicio                        1332 non-null   datetime64[ns]
 2   fecha_fin                           1332 non-null   datetime64[ns]
 3   dias                                1332 non-null   int64         
 4   semana                              1332 non-null   int64         
 5   total_muertes_reportadas            1332 non-null   int64         
 6   total_muertes_reportadas_por_covid  1332 non-null   int64         
dtypes: datetime64[ns](2), int64(4), object(1)
memory usage: 73.0+ KB


In [160]:
for column in df_muertes_covid.select_dtypes(include=['object', 'datetime64[ns]']).columns:
    print(f"{column}: {df_muertes_covid[column].nunique()} valores únicos")

pais: 5 valores únicos
fecha_inicio: 428 valores únicos
fecha_fin: 428 valores únicos


In [161]:
df_muertes_covid.head()

,pais,fecha_inicio,fecha_fin,dias,semana,total_muertes_reportadas,total_muertes_reportadas_por_covid
0,Mexico,2014-12-29,2015-01-04,7,1,15101,0
1,Mexico,2015-01-05,2015-01-11,7,2,15449,0
2,Mexico,2015-01-12,2015-01-18,7,3,14820,0
3,Mexico,2015-01-19,2015-01-25,7,4,14172,0
4,Mexico,2015-01-26,2015-02-01,7,5,13628,0


In [162]:
df_muertes_covid.describe().T

,count,mean,std,min,25%,50%,75%,max
dias,1332.0,8.424925,5.605587,7.0,7.0,7.0,7.0,31.0
semana,1332.0,24.669670,15.193435,1.0,11.0,24.0,38.0,53.0
total_muertes_reportadas,1332.0,13630.075075,27946.362998,1586.0,2292.5,4433.5,12970.0,205125.0
total_muertes_reportadas_por_covid,1332.0,920.036036,4573.413708,0.0,0.0,0.0,25.5,82266.0


**Conclusão**
A base *df_muertes_covid* está sem erros como nulos ou valores incoerentes.

### Criando novos atributos

A partir do atributo *fecha_inicio*, vou contruir os seguintes atributos derivados desse atributo de data:

1. fecha_inicio_ano
1. fecha_inicio_ano_mes

In [168]:
# Criação dos atributos:

df_muertes_covid['fecha_inicio_ano'] = df_muertes_covid['fecha_inicio'].dt.year
df_muertes_covid['fecha_inicio_ano_mes'] = df_muertes_covid['fecha_inicio'].dt.year.astype(str) + df_muertes_covid['fecha_inicio'].dt.month.astype(str).str.zfill(2)
df_muertes_covid.head()

,pais,fecha_inicio,fecha_fin,dias,semana,total_muertes_reportadas,total_muertes_reportadas_por_covid,fecha_inicio_ano,fecha_inicio_ano_mes
0,Mexico,2014-12-29,2015-01-04,7,1,15101,0,2014,201412
1,Mexico,2015-01-05,2015-01-11,7,2,15449,0,2015,201501
2,Mexico,2015-01-12,2015-01-18,7,3,14820,0,2015,201501
3,Mexico,2015-01-19,2015-01-25,7,4,14172,0,2015,201501
4,Mexico,2015-01-26,2015-02-01,7,5,13628,0,2015,201501


In [172]:
# Criação da tabela agregada por ano por país

df_muertes_covid_agg_yearly = df_muertes_covid.groupby(['pais', 'fecha_inicio_ano'])[[
    'total_muertes_reportadas', 'total_muertes_reportadas_por_covid'
    ]].agg({'total_muertes_reportadas': 'sum', 'total_muertes_reportadas_por_covid': 'sum'}).reset_index()

df_muertes_covid_agg_yearly.head(8)

,pais,fecha_inicio_ano,total_muertes_reportadas,total_muertes_reportadas_por_covid
0,Brazil,2015,1264175,0
1,Brazil,2016,1309774,0
2,Brazil,2017,1312663,0
3,Brazil,2018,1316719,0
4,Brazil,2019,1349802,0
5,Brazil,2020,1581645,194949
6,Brazil,2021,1491943,401800
7,Chile,2016,103375,0


In [164]:
# Criação da tabela agregada por ano e mês por país

df_muertes_covid_agg_monthly = df_muertes_covid.groupby(['pais', 'fecha_inicio_ano_mes'])[[
    'total_muertes_reportadas', 'total_muertes_reportadas_por_covid'
    ]].agg({'total_muertes_reportadas': 'sum', 'total_muertes_reportadas_por_covid': 'sum'}).reset_index()

df_muertes_covid_agg_monthly.head()

,pais,fecha_inicio_ano_mes,total_muertes_reportadas,total_muertes_reportadas_por_covid
0,Brazil,201501,106695,0
1,Brazil,201502,90612,0
2,Brazil,201503,102473,0
3,Brazil,201504,105114,0
4,Brazil,201505,114042,0


In [165]:
df_muertes_covid_agg_country = df_muertes_covid.groupby(['pais'])[[
    'total_muertes_reportadas', 'total_muertes_reportadas_por_covid'
    ]].agg({'total_muertes_reportadas': 'sum', 'total_muertes_reportadas_por_covid': 'sum'}).reset_index()
    
df_muertes_covid_agg_country

,pais,total_muertes_reportadas,total_muertes_reportadas_por_covid
0,Brazil,9626721,596749
1,Chile,665527,37669
2,Colombia,1672460,112826
3,Mexico,5426848,277814
4,Peru,763704,200430


### Juntando as informações das tabelas *df_muertes_covid_agg_country* e *df_poblacion*

In [169]:
print(len(df_muertes_covid_agg_country))
print(len(df_poblacion))
print(len(df_muertes_covid_agg_country.merge(df_poblacion, how='left', on='pais')))

df_muertes_covid_agg_country_merged = df_muertes_covid_agg_country.merge(df_poblacion, how='left', on='pais')
df_muertes_covid_agg_country_merged

5
5
5


,pais,total_muertes_reportadas,total_muertes_reportadas_por_covid,poblacion
0,Brazil,9626721,596749,213993441
1,Chile,665527,37669,19212362
2,Colombia,1672460,112826,51265841
3,Mexico,5426848,277814,130262220
4,Peru,763704,200430,33359415
